# The generative function interface [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChiSym/genjax/blob/main/docs/cookbook/active/generative_function_interface.ipynb)

In [1]:
import sys

if "google.colab" in sys.modules:
    %pip install --quiet "genjax[genstudio]"

In [2]:
import jax
from jax import jit

from genjax import ChoiceMapBuilder as C
from genjax import (
    bernoulli,
    beta,
    gen,
    pretty,
)
from genjax._src.generative_functions.static import MissingAddress
from genjax.incremental import Diff, NoChange, UnknownChange

key = jax.random.key(0)
pretty()


# Define a generative function
@gen
def beta_bernoulli_process(u):
    p = beta(1.0, u) @ "p"
    v = bernoulli(p) @ "v"
    return 2 * v

1) Generate a traced sample and constructs choicemaps

There's an entire cookbook entry on this in `choicemap_creation_selection`.

In [3]:
key, subkey = jax.random.split(key)
trace = jax.jit(beta_bernoulli_process.simulate)(subkey, (0.5,))

2) Compute log probabilities

2.1 Print the log probability of the trace

In [4]:
trace.get_score()

<jax.Array(-1.1125305, dtype=float32)>

2.2 Print the log probability of an observation encoded as a ChoiceMap under the model

It returns both the log probability and the return value

In [5]:
chm = C["p"].set(0.5) ^ C["v"].set(1)
args = (0.5,)
beta_bernoulli_process.assess(chm, args)

/var/folders/kj/tyb60krx2jz817l0x_3v4j680000gn/T/ipykernel_18597/3994869088.py:1: DeprecationWarning: Call to deprecated method __xor__. (^ is deprecated, please use | or _.merge(...) instead.) -- Deprecated since version 0.8.0.
  chm = C["p"].set(0.5) ^ C["v"].set(1)


(<jax.Array(-0.8206506, dtype=float32)>, <jax.Array(2, dtype=int32)>)

Note that the ChoiceMap should be complete, i.e. all random choices should be observed

In [6]:
chm_2 = C["v"].set(1)
try:
    beta_bernoulli_process.assess(chm_2, args)
except MissingAddress as e:
    print(e)

p


3) Generate a sample conditioned on the observations

We can also use a partial ChoiceMap as a constraint/observation and generate a full trace with these constraints.

In [7]:
key, subkey = jax.random.split(key)
partial_chm = C["v"].set(1)  # Creates a ChoiceMap of observations
args = (0.5,)
trace, weight = beta_bernoulli_process.importance(
    subkey, partial_chm, args
)  # Runs importance sampling

This returns a pair containing the new trace and the log probability of produced trace under the model

In [8]:
trace.get_choices()

Static({
  'v': Choice(v=1),
  'p': Choice(v=<jax.Array(0.9927626, dtype=float32)>),
})

In [9]:
weight

<jax.Array(-0.31521326, dtype=float32)>

4) Update a trace.

We can also update a trace. This is for instance useful as a performance optimization in Metropolis-Hastings algorithms where often most of the trace doesn't change between time steps.

We first define a model for which changing the argument will force a change in the trace.

In [10]:
@gen
def beta_bernoulli_process(u):
    p = beta(1.0, u) @ "p"
    v = bernoulli(p) @ "v"
    return 2 * v

We then create an trace to be updated and constraints.

In [11]:
key, subkey = jax.random.split(key)
jitted = jit(beta_bernoulli_process.simulate)
old_trace = jitted(subkey, (1.0,))
constraint = C["v"].set(1)

Now the update uses a form of incremental computation.
It works by tracking the differences between the old new values for arguments.
Just like for differentiation, it can be achieved by providing for each argument a tuple containing the new value and its change compared to the old value.

If there's no change for an argument, the change is set to NoChange.

In [12]:
arg_diff = (Diff(1.0, NoChange),)

If there is any change, the change is set to UnknownChange.

In [13]:
arg_diff = (Diff(5.0, UnknownChange),)

We finally use the update method on the trace by passing it a key, and the update to be performed.

In [14]:
key, subkey = jax.random.split(key)
new_trace, weight_diff, ret_diff, discard_choice = jit(old_trace.update)(
    subkey, constraint, arg_diff
)

TypeError: Method[1m[35m genjax._src.core.generative.generative_function.Trace.update()[0m parameter [1m[34mconstraint[0m=[1m[31m"StaticTrace(gen_fn=StaticGenerativeFunction(source=Closure(dyn_args=(), fn=<function be...>)})"[0m violates type hint [1m[32m<class 'genjax._src.core.generative.choice_map.ChoiceMap'>[0m, as [1m[33m<class "genjax._src.generative_functions.static.StaticTrace"> [0m[1m[31m"StaticTrace(gen_fn=StaticGenerativeFunction(source=Closure(dyn_args=(), fn=<function be...>)})"[0m not instance of [1m[33m<class "genjax._src.core.generative.choice_map.ChoiceMap">[0m.

We can compare the old and new values for the samples and notice that they have not changed.

In [ ]:
old_trace.get_choices() == new_trace.get_choices()

We can also see that the weight has changed. In fact we can check that the following relation holds `new_weight` = `old_weight` + `weight_diff`.

In [ ]:
weight_diff, old_trace.get_score() + weight_diff == new_trace.get_score()

   5. A few more convenient methods

5.1 `propose`

It uses the same inputs as `simulate` but returns the sample, the score and the return value

In [ ]:
key, subkey = jax.random.split(key)
sample, score, retval = jit(beta_bernoulli_process.propose)(subkey, (0.5,))
sample, score, retval

5.2 `get_gen_fn`

It returns the generative function that produced the trace.

In [ ]:
trace.get_gen_fn()

5.3 `get_args`

It returns the arguments passed to the generative function used to produce the trace

In [ ]:
trace.get_args()

5.4 `get_subtrace`

It takes a `StaticAddress` as argument and returns the sub-trace of a trace rooted at these addresses

In [ ]:
subtrace = trace.get_subtrace("p")
subtrace, subtrace.get_choices()